# Dimensionality reduction

### Prepare the data

In [1]:
from sklearn import datasets
data_breast_cancer = datasets.load_breast_cancer()
data_iris = datasets.load_iris()

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [3]:
# PCA for breast_cancer dataset

X_bc = data_breast_cancer.data
y_bc = data_breast_cancer.target
# we create PCA object to cover 90% of the variation
pca_bc = PCA(n_components=0.9)
X_bc90 = pca_bc.fit_transform(X_bc)
print(pca_bc.explained_variance_ratio_)
# Now we scale the data to get better results
X_bc_sc = StandardScaler().fit_transform(X_bc)
pca_bc_sc = PCA(n_components=0.9)
X_bc90_sc = pca_bc_sc.fit_transform(X_bc_sc)
print(pca_bc_sc.explained_variance_ratio_)

print(f"Redukcja wymiarowości: {X_bc.shape} => {X_bc90_sc.shape}")

[0.98204467]
[0.44272026 0.18971182 0.09393163 0.06602135 0.05495768 0.04024522
 0.02250734]
Redukcja wymiarowości: (569, 30) => (569, 7)


Możemy zawuażyć, że nieprzeskalowane dane redukują się do 1 wymiaru o bardzo dużej wariancji. Może to wynikać z faktu, że niektóre cechy mają dużo większą skalę od innych i model "sztucznie" zawyża wariancję dla nich. W przypadku przeskalowanych cech dostajemy już bardziej wiarygodne wyniki.

In [4]:
# iris dataset
X_ir = data_iris.data
y_ir = data_iris.target
pca_ir = PCA(n_components=0.9)
X_ir90 = pca_ir.fit_transform(X_ir)
print(pca_ir.explained_variance_ratio_)
print(f"Redukcja wymiarowości: (nieprzeskalowana) {X_ir.shape} => {X_ir90.shape}")
# Scale the data to get better results
X_ir_sc = StandardScaler().fit_transform(X_ir)
pca_ir_sc = PCA(n_components=0.9)
X_ir90_sc = pca_ir_sc.fit_transform(X_ir_sc)
print(pca_ir_sc.explained_variance_ratio_)
print(f"Redukcja wymiarowości: {X_ir_sc.shape} => {X_ir90_sc.shape}")

[0.92461872]
Redukcja wymiarowości: (nieprzeskalowana) (150, 4) => (150, 1)
[0.72962445 0.22850762]
Redukcja wymiarowości: (150, 4) => (150, 2)


Znowu przeskalowane dane wydają się dawać bardziej sensowne wyniki

### Saving the explained variance ratio to pickle file

In [5]:
import pickle

with open("pca_bc.pkl", "wb") as file:
  pickle.dump(pca_bc_sc.explained_variance_ratio_, file)

with open("pca_ir.pkl", "wb") as file:
  pickle.dump(pca_ir_sc.explained_variance_ratio_, file)

### Calculating which features (indexes) have the highest infuence to our variance

*pca.components_* zwraca wektory własne macierzy kowariancji, czyli **główne składowe**

In [6]:
print(pca_ir_sc.explained_variance_ratio_[:, np.newaxis])

[[0.72962445]
 [0.22850762]]


In [19]:
# components_ is 2D array so we have to reshape our variance_ratio array from 1D to 2D
weighted_components = np.abs(pca_bc_sc.components_ * pca_bc_sc.explained_variance_ratio_.reshape(-1, 1))
sorted_index = np.argsort(np.max(weighted_components, axis=0))[::-1] #[::-1] to change the order
sorted_index = list(dict.fromkeys(sorted_index))

with open("idx_bc.pkl", "wb") as file:
  pickle.dump(sorted_index, file)
print(sorted_index)

[np.int64(7), np.int64(6), np.int64(27), np.int64(5), np.int64(22), np.int64(26), np.int64(20), np.int64(2), np.int64(23), np.int64(3), np.int64(0), np.int64(12), np.int64(25), np.int64(10), np.int64(13), np.int64(17), np.int64(15), np.int64(9), np.int64(16), np.int64(4), np.int64(8), np.int64(29), np.int64(24), np.int64(28), np.int64(19), np.int64(21), np.int64(1), np.int64(14), np.int64(11), np.int64(18)]


In [20]:
weighted_components = np.abs(pca_ir_sc.components_ * pca_ir_sc.explained_variance_ratio_.reshape(-1, 1))
sorted_index = np.argsort(np.max(weighted_components, axis=0)[::-1]) #[::-1] to change the order
sorted_index = list(dict.fromkeys(sorted_index))

with open("idx_ir.pkl", "wb") as file:
  pickle.dump(sorted_index, file)
print(sorted_index)

[np.int64(2), np.int64(3), np.int64(0), np.int64(1)]
